# For when I wanna edit non code parts
Neither is this part. 
Press esc+1 to become code. 
Press ctrl+return to leave the cell

# Plan for the rest of the notebook
    -Track all of the actions data from the inside a dictionary
    -Plot all of the actions by coordinates (Shots ---> Done)
        -Figure out a way to plot them on an xy grid (For one game ---> Done)
        -Render an ice rink on top of the xy? ("rendered" ---> Done)
            -Place a net if you can't render
            -Plot the lines with specific colors in the xy if not possible
    -Track the angle towards the respective net
    -And track the distance from the location of the shot towards the net 

In [2]:
class Missed_shot: #holds missed_shots
    player = ''
    _x = 0
    _y = 0
    team = ''
    period = 0
    time = ''
    shot_type = ''

class On_Net_Shot(Missed_shot):
    goalie = ''
    
class Blocked_Shot(Missed_shot):
    blockee = ''
    
class Goal(On_Net_Shot):
    assists = None #List of players who assisted

In [3]:
from bokeh.io import output_notebook, show
import math
output_notebook()


Loading BokehJS ...

In [4]:
import json, urllib.request
import numpy as np
import pandas as pd

#Schedule for the 2018-2019 regular season: https://statsapi.web.nhl.com/api/v1/schedule?startDate=2018-09-01&endDate=2019-06-14&hydrate=team(leaders(categories=[points,goals,assists],gameTypes=[P])),linescore,broadcasts(all),tickets,game(content(media(epg),highlights(scoreboard)),seriesSummary),radioBroadcasts,metadata,seriesSummary(series),decisions,scoringplays&leaderCategories=&site=en_nhl&teamId=&gameType=&timecode=

'''
for j in range(5): #running through the schedule json
    #loading in the goalie data
    Game_IDs = 'https://statsapi.web.nhl.com/api/v1/schedule?startDate=2017-10-4&endDate=2018-06-10&hydrate=team(leaders(categories=[points,goals,assists],gameTypes=[P])),linescore,broadcasts(all),tickets,game(content(media(epg),highlights(scoreboard)),seriesSummary),radioBroadcasts,metadata,seriesSummary(series),decisions,scoringplays&leaderCategories=&site=en_nhl&teamId=&gameType=&timecode='
    #GameData = "http://statsapi.web.nhl.com/api/v1/game/2017020001/feed/live"
    
    for k in range(2): #checking inside the games on a specific date -- json
        game_id = Game_IDs['dates'][j]['games'][k]['gamePk']
        GameData = findGame(game_id)
        with urllib.request.urlopen(GameData) as url:  # getting the info into a dictionary
            game_data = json.loads(url.read().decode())
'''     

#GameData = "http://statsapi.web.nhl.com/api/v1/game/2017020001/feed/live" #winnipeg vs toronto
GameData = "https://statsapi.web.nhl.com/api/v1/game/2017030311/feed/live"
with urllib.request.urlopen(GameData) as url:  # getting the info into a dictionary
            game_data = json.loads(url.read().decode())

# Shots ############
player_1 = []
player_2 = []
team_1 = []
team_2 = []
x_1 = []
y_1 = []
x_2 = []
y_2 = []
period_1 = []
period_2 = []
time_1 = []
time_2 = []
shot_type_1 = []
shot_type_2 = []
####################

on_net_shot_list_1 = []
goal_list_1 = []
blocked_shot_list_1 = []
missed_shot_list_1 = []

on_net_shot_list_2 = []
goal_list_2 = []
blocked_shot_list_2 = []
missed_shot_list_2 = []

homeTeam = game_data['gameData']['teams']['home']['name']
awayTeam = game_data['gameData']['teams']['away']['name']

for i in range(len(game_data['liveData']['plays']['allPlays'])): #checking the game json
    if game_data['liveData']['plays']['allPlays'][i]['result']['eventTypeId'] == 'GOAL':
        player = game_data['liveData']['plays']['allPlays'][i]['players'][0]['player']['fullName']
        team = game_data['liveData']['plays']['allPlays'][i]['team']['name']
        _x = game_data['liveData']['plays']['allPlays'][i]['coordinates']['x'] + 100
        _y = game_data['liveData']['plays']['allPlays'][i]['coordinates']['y'] + 42.5
        period =  game_data['liveData']['plays']['allPlays'][i]['about']['period']
        time =  game_data['liveData']['plays']['allPlays'][i]['about']['periodTime']
        type_of_shot = game_data['liveData']['plays']['allPlays'][i]['result']['secondaryType']
        if (team == homeTeam):
            player_1.append(player)        
            team_1.append(team)
            x_1.append(_x)
            y_1.append(_y) 
            period_1.append(period)
            time_1.append(time)
            shot_type_1.append(type_of_shot)
        else:
            player_2.append(player)
            team_2.append(team)
            x_2.append(_x)
            y_2.append(_y)     
            period_2.append(period)
            time_2.append(time)
            shot_type_2.append(type_of_shot)

   #elif game_data['liveData']['plays']['allPlays'][i]['result']['eventTypeId'] == 'GOAL':

df_coord_1 = pd.DataFrame(data = dict(x = x_1, y = y_1))
df_coord_2 = pd.DataFrame(data = dict(x = x_2, y = y_2))
df_player_1 = pd.DataFrame(data = dict(player = player_1, team = team_1))
df_player_2 = pd.DataFrame(data = dict(player = player_2, team = team_2))
df_clock_1 = pd.DataFrame(data = dict(period = period_1, time = time_1))
df_clock_2 = pd.DataFrame(data = dict(period = period_2, time = time_2))
df_angle_1 = pd.DataFrame(data = dict())
#df_shot_type = pd.DataFrame(data = dict(shot_type_1 = shot_type_1, shot_type_2 = shot_type_2))


In [5]:
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.plotting import figure
from bokeh.sampledata.autompg import autompg_clean as df
from bokeh.transform import factor_cmap

p = figure(plot_height = 400, plot_width = 940, title="Shot Quality")

#making the multiline for the lines
#the first circle is for the dots

#Setting up the rink #############################
p.circle(x = [11, 189], y = [42.5, 42.5], size = 50, fill_color = 'white')
p.circle(x = [11, 189], y = [42.5, 42.5], size = 5, color = 'firebrick')
p.circle(x = [100, 169, 169, 41, 41], 
         y = [42.5, 64.5, 22.5, 64.5, 22.5],
         size = 100, fill_color = 'white')
p.circle(x = [100, 169, 169, 41, 41, 80, 80, 120, 120], 
         y = [42.5, 64.5, 22.5, 64.5, 22.5, 64.5, 20.5, 64.5, 20.5],
         size = 10, color = 'firebrick')
p.multi_line([[11,11], [189, 189], [75,75], [125, 125], [100,100], [0,200], [0,200], [0,0], [200,200]], 
             [[0,85], [0,85], [0,85], [0,85], [0,85], [0,0], [85, 85], [0,85], [0,85]], 
             line_width = 2, color = ['firebrick', 'firebrick', 'blue', 'blue', 'firebrick', 'black', 'black', 'black', 'black'])
####################################################### 

#Plotting the shots #############################

source_1 = ColumnDataSource(data = dict(x=df_coord_1['x'].values,
                                      y=df_coord_1['y'].values,
                                     Player= df_player_1['player'].values,
                                     Team= df_player_1['team'].values,
                                       period = df_clock_1['period'].values,
                                       time = df_clock_1['time'].values,
                                       shots = shot_type_1))

source_2 = ColumnDataSource(data = dict(x=df_coord_2['x'].values,
                                      y=df_coord_2['y'].values,
                                     Player= df_player_2['player'].values,
                                     Team= df_player_2['team'].values,
                                        period = df_clock_2['period'].values,
                                       time = df_clock_2['time'].values,
                                       shots = shot_type_2))


shots_1 = p.circle(x = 'x', y = 'y', size = 8, color = 'orange', fill_color = 'white', source = source_1, line_width = 2)
shots_2 = p.circle(x = 'x', y = 'y', size = 8, color = 'blue', fill_color = 'white', source = source_2, line_width = 2)

shots_hover = HoverTool(renderers = [shots_1, shots_2], 
                        tooltips = [("X", "@x"), ("Y", "@y"), ("Name", "@Player"), ("Shot Type", "@shots"), 
                                    ("Team", "@Team"),
                                   ("Period", "@period"), ("Time", "@time")])
p.add_tools(shots_hover)
#################################################

show(p)
#Orange is home 
#Blue if away